# 🌐 Reality Mesh AI - Spatial Understanding Processor

This notebook processes data captured from the Reality Mesh AI app and builds spatial understanding models.

## Features:
- 📊 Load and parse Reality Mesh sensor + point cloud data
- 🧭 Sensor fusion (GPS, gyro, accel, magnetometer)
- 🗺️ 3D point cloud reconstruction
- 📐 Spatial mapping and localization
- 🎨 Interactive 3D visualization
- 🤖 Export for AI/ML training

## Upload Your Data:
1. Capture data using: https://ibreitenbach.github.io/reality-mesh-ai/
2. Export the JSON file
3. Upload it to this Colab session

In [ ]:
# Install required packages
!pip install -q numpy pandas matplotlib plotly opencv-python scipy scikit-learn

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
from sklearn.cluster import DBSCAN
from google.colab import files
import io

## Step 1: Upload Reality Mesh Data

In [ ]:
# Upload JSON file from Reality Mesh app
print("📁 Upload your reality-mesh-sensor-data-*.json file:")
uploaded = files.upload()

# Load the first uploaded file
filename = list(uploaded.keys())[0]
print(f"\n✅ Loaded: {filename}")

with open(filename, 'r') as f:
    data = json.load(f)

print("\n📊 Data Summary:")
print(f"  Frames captured: {data['metadata']['frameCount']}")
print(f"  Point cloud size: {data['metadata']['pointCount']}")
print(f"  Sensor samples: {data['metadata']['sensorSamples']}")
print(f"  Duration: {data['metadata']['duration']:.2f} seconds")
print(f"  Device: {data['metadata']['device'][:50]}...")

## Step 2: Parse Sensor Data

In [ ]:
# Extract sensor data into DataFrames
sensor_records = []

for sample in data['sensors']:
    record = {
        'frame': sample['frame'],
        'timestamp': sample['timestamp'],
        # Orientation
        'orient_alpha': float(sample['orientation']['alpha']),
        'orient_beta': float(sample['orientation']['beta']),
        'orient_gamma': float(sample['orientation']['gamma']),
        # Gyroscope
        'gyro_x': float(sample['gyroscope']['x']),
        'gyro_y': float(sample['gyroscope']['y']),
        'gyro_z': float(sample['gyroscope']['z']),
        # Accelerometer
        'accel_x': float(sample['accelerometer']['x']),
        'accel_y': float(sample['accelerometer']['y']),
        'accel_z': float(sample['accelerometer']['z']),
        # GPS
        'gps_lat': float(sample['gps']['lat']),
        'gps_lon': float(sample['gps']['lon']),
        'gps_alt': float(sample['gps']['alt']),
        'gps_accuracy': float(sample['gps']['accuracy']),
        # Light
        'light': float(sample['light']) if sample['light'] != 'N/A' else 0
    }
    sensor_records.append(record)

df_sensors = pd.DataFrame(sensor_records)
df_sensors['time'] = (df_sensors['timestamp'] - df_sensors['timestamp'].min()) / 1000  # Convert to seconds

print("\n📊 Sensor DataFrame:")
print(df_sensors.head())
print(f"\nShape: {df_sensors.shape}")

## Step 3: Parse Point Cloud Data

In [ ]:
# Extract point cloud
points = []
colors = []

for point in data['pointCloud']:
    points.append([point['x'], point['y'], point['z']])
    colors.append([point['r'], point['g'], point['b']])

points = np.array(points)
colors = np.array(colors)

print(f"\n☁️ Point Cloud:")
print(f"  Total points: {len(points):,}")
print(f"  X range: [{points[:,0].min():.2f}, {points[:,0].max():.2f}]")
print(f"  Y range: [{points[:,1].min():.2f}, {points[:,1].max():.2f}]")
print(f"  Z range: [{points[:,2].min():.2f}, {points[:,2].max():.2f}]")

## Step 4: Sensor Fusion & Pose Estimation

In [ ]:
def compute_device_poses(df):
    """
    Compute device pose (position + orientation) from sensor data
    """
    poses = []
    
    # Initial position (from GPS)
    base_lat = df['gps_lat'].iloc[0]
    base_lon = df['gps_lon'].iloc[0]
    base_alt = df['gps_alt'].iloc[0]
    
    for i, row in df.iterrows():
        # Convert GPS to relative meters (approximate)
        lat_diff = (row['gps_lat'] - base_lat) * 111320  # meters per degree latitude
        lon_diff = (row['gps_lon'] - base_lon) * 111320 * np.cos(np.radians(base_lat))
        alt_diff = row['gps_alt'] - base_alt
        
        # Orientation as rotation matrix
        r = Rotation.from_euler('zyx', [
            np.radians(row['orient_alpha']),
            np.radians(row['orient_beta']),
            np.radians(row['orient_gamma'])
        ])
        
        poses.append({
            'frame': row['frame'],
            'time': row['time'],
            'position': np.array([lon_diff, lat_diff, alt_diff]),
            'rotation': r,
            'rotation_matrix': r.as_matrix()
        })
    
    return poses

poses = compute_device_poses(df_sensors)
print(f"\n📍 Computed {len(poses)} device poses")
print(f"  Movement range: {np.linalg.norm([p['position'] for p in poses], axis=1).max():.2f} meters")

## Step 5: Visualize Sensor Data

In [ ]:
# Plot sensor readings over time
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Orientation
axes[0,0].plot(df_sensors['time'], df_sensors['orient_alpha'], label='Alpha (Yaw)')
axes[0,0].plot(df_sensors['time'], df_sensors['orient_beta'], label='Beta (Pitch)')
axes[0,0].plot(df_sensors['time'], df_sensors['orient_gamma'], label='Gamma (Roll)')
axes[0,0].set_title('Device Orientation')
axes[0,0].set_xlabel('Time (s)')
axes[0,0].set_ylabel('Degrees')
axes[0,0].legend()
axes[0,0].grid(True)

# Gyroscope
axes[0,1].plot(df_sensors['time'], df_sensors['gyro_x'], label='X')
axes[0,1].plot(df_sensors['time'], df_sensors['gyro_y'], label='Y')
axes[0,1].plot(df_sensors['time'], df_sensors['gyro_z'], label='Z')
axes[0,1].set_title('Gyroscope (Rotation Rate)')
axes[0,1].set_xlabel('Time (s)')
axes[0,1].set_ylabel('deg/s')
axes[0,1].legend()
axes[0,1].grid(True)

# Accelerometer
axes[1,0].plot(df_sensors['time'], df_sensors['accel_x'], label='X')
axes[1,0].plot(df_sensors['time'], df_sensors['accel_y'], label='Y')
axes[1,0].plot(df_sensors['time'], df_sensors['accel_z'], label='Z')
axes[1,0].set_title('Accelerometer')
axes[1,0].set_xlabel('Time (s)')
axes[1,0].set_ylabel('m/s²')
axes[1,0].legend()
axes[1,0].grid(True)

# GPS trajectory
axes[1,1].plot(df_sensors['gps_lon'], df_sensors['gps_lat'], 'b-', marker='o', markersize=2)
axes[1,1].set_title('GPS Trajectory')
axes[1,1].set_xlabel('Longitude')
axes[1,1].set_ylabel('Latitude')
axes[1,1].grid(True)

# Altitude
axes[2,0].plot(df_sensors['time'], df_sensors['gps_alt'])
axes[2,0].set_title('Altitude')
axes[2,0].set_xlabel('Time (s)')
axes[2,0].set_ylabel('Meters')
axes[2,0].grid(True)

# Light sensor
axes[2,1].plot(df_sensors['time'], df_sensors['light'])
axes[2,1].set_title('Ambient Light')
axes[2,1].set_xlabel('Time (s)')
axes[2,1].set_ylabel('Lux')
axes[2,1].grid(True)

plt.tight_layout()
plt.show()

print("\n✅ Sensor data visualized")

## Step 6: 3D Point Cloud Visualization

In [ ]:
# Interactive 3D point cloud visualization with Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=['rgb({},{},{})'.format(int(r*255), int(g*255), int(b*255)) 
               for r, g, b in colors],
        opacity=0.8
    )
)])

fig.update_layout(
    title='Reality Mesh Point Cloud',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z (Depth)',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig.show()
print("\n✅ Point cloud rendered")

## Step 7: Spatial Clustering & Object Detection

In [ ]:
# Apply DBSCAN clustering to detect spatial objects
print("🔍 Running spatial clustering...")
clustering = DBSCAN(eps=0.5, min_samples=10).fit(points)
labels = clustering.labels_

# Number of clusters
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = list(labels).count(-1)

print(f"\n📦 Spatial Clustering Results:")
print(f"  Clusters found: {n_clusters}")
print(f"  Noise points: {n_noise}")

# Visualize clusters
fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=labels,
        colorscale='Viridis',
        opacity=0.8,
        showscale=True
    )
)])

fig.update_layout(
    title=f'Spatial Clusters ({n_clusters} objects detected)',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig.show()

## Step 8: Build Spatial Understanding Model

In [ ]:
class SpatialUnderstandingModel:
    """
    Spatial understanding model that combines:
    - Point cloud geometry
    - Device trajectory
    - Sensor context (light, orientation, etc.)
    """
    def __init__(self, points, colors, poses, sensors):
        self.points = points
        self.colors = colors
        self.poses = poses
        self.sensors = sensors
        
    def compute_spatial_features(self):
        """Extract spatial features for AI/ML"""
        features = {
            # Geometry features
            'point_cloud_size': len(self.points),
            'spatial_extent': {
                'x_range': (self.points[:,0].min(), self.points[:,0].max()),
                'y_range': (self.points[:,1].min(), self.points[:,1].max()),
                'z_range': (self.points[:,2].min(), self.points[:,2].max())
            },
            'centroid': self.points.mean(axis=0).tolist(),
            
            # Trajectory features
            'num_poses': len(self.poses),
            'total_distance': sum([
                np.linalg.norm(self.poses[i]['position'] - self.poses[i-1]['position'])
                for i in range(1, len(self.poses))
            ]),
            
            # Sensor statistics
            'avg_light': self.sensors['light'].mean(),
            'orientation_variance': {
                'alpha': self.sensors['orient_alpha'].std(),
                'beta': self.sensors['orient_beta'].std(),
                'gamma': self.sensors['orient_gamma'].std()
            },
            'gps_accuracy': self.sensors['gps_accuracy'].mean()
        }
        
        return features
    
    def export_for_training(self):
        """Export data in format suitable for ML training"""
        return {
            'geometry': {
                'points': self.points.tolist(),
                'colors': self.colors.tolist()
            },
            'trajectory': [
                {
                    'frame': p['frame'],
                    'position': p['position'].tolist(),
                    'rotation_matrix': p['rotation_matrix'].tolist()
                }
                for p in self.poses
            ],
            'sensors': self.sensors.to_dict('records'),
            'features': self.compute_spatial_features()
        }

# Create model
model = SpatialUnderstandingModel(points, colors, poses, df_sensors)
features = model.compute_spatial_features()

print("\n🧠 Spatial Understanding Features:")
print(json.dumps(features, indent=2, default=str))

## Step 9: Device Trajectory Visualization

In [ ]:
# Extract trajectory positions
trajectory = np.array([p['position'] for p in poses])

# Create 3D trajectory plot with orientation arrows
fig = go.Figure()

# Add trajectory line
fig.add_trace(go.Scatter3d(
    x=trajectory[:, 0],
    y=trajectory[:, 1],
    z=trajectory[:, 2],
    mode='lines+markers',
    marker=dict(size=5, color='red'),
    line=dict(color='red', width=3),
    name='Device Path'
))

# Add start and end points
fig.add_trace(go.Scatter3d(
    x=[trajectory[0, 0]],
    y=[trajectory[0, 1]],
    z=[trajectory[0, 2]],
    mode='markers',
    marker=dict(size=10, color='green', symbol='diamond'),
    name='Start'
))

fig.add_trace(go.Scatter3d(
    x=[trajectory[-1, 0]],
    y=[trajectory[-1, 1]],
    z=[trajectory[-1, 2]],
    mode='markers',
    marker=dict(size=10, color='blue', symbol='diamond'),
    name='End'
))

fig.update_layout(
    title='Device Trajectory (GPS-based)',
    scene=dict(
        xaxis_title='East-West (m)',
        yaxis_title='North-South (m)',
        zaxis_title='Altitude (m)',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig.show()

print(f"\n📍 Total distance traveled: {features['total_distance']:.2f} meters")

## Step 10: Export Processed Data for AI Training

In [ ]:
# Export complete spatial understanding dataset
training_data = model.export_for_training()

# Save to file
output_filename = 'spatial_understanding_training_data.json'
with open(output_filename, 'w') as f:
    json.dump(training_data, f, indent=2)

print(f"\n💾 Training data exported to: {output_filename}")
print(f"\n📊 Dataset Summary:")
print(f"  Point cloud: {len(training_data['geometry']['points']):,} points")
print(f"  Trajectory: {len(training_data['trajectory'])} poses")
print(f"  Sensor samples: {len(training_data['sensors'])}")
print(f"\n✅ Ready for AI/ML model training!")

# Download the file
files.download(output_filename)

## Step 11: Generate Spatial Understanding Report

In [ ]:
print("\n" + "="*60)
print("📋 SPATIAL UNDERSTANDING REPORT")
print("="*60)

print(f"\n🎥 Capture Session:")
print(f"  Duration: {data['metadata']['duration']:.2f} seconds")
print(f"  Frames: {data['metadata']['frameCount']}")
print(f"  Device: {data['metadata']['device'][:50]}")

print(f"\n☁️ Point Cloud:")
print(f"  Total points: {len(points):,}")
print(f"  Spatial extent:")
print(f"    X: {features['spatial_extent']['x_range'][0]:.2f} to {features['spatial_extent']['x_range'][1]:.2f}")
print(f"    Y: {features['spatial_extent']['y_range'][0]:.2f} to {features['spatial_extent']['y_range'][1]:.2f}")
print(f"    Z: {features['spatial_extent']['z_range'][0]:.2f} to {features['spatial_extent']['z_range'][1]:.2f}")
print(f"  Centroid: ({features['centroid'][0]:.2f}, {features['centroid'][1]:.2f}, {features['centroid'][2]:.2f})")

print(f"\n📍 Device Trajectory:")
print(f"  Poses captured: {features['num_poses']}")
print(f"  Total distance: {features['total_distance']:.2f} meters")
print(f"  GPS accuracy: {features['gps_accuracy']:.1f} meters")

print(f"\n🧭 Orientation Variance:")
print(f"  Alpha (Yaw): {features['orientation_variance']['alpha']:.2f}°")
print(f"  Beta (Pitch): {features['orientation_variance']['beta']:.2f}°")
print(f"  Gamma (Roll): {features['orientation_variance']['gamma']:.2f}°")

print(f"\n💡 Environment:")
print(f"  Avg ambient light: {features['avg_light']:.0f} lux")

if n_clusters > 0:
    print(f"\n📦 Spatial Objects:")
    print(f"  Detected clusters: {n_clusters}")
    print(f"  Noise points: {n_noise:,}")

print("\n" + "="*60)
print("✅ SPATIAL UNDERSTANDING COMPLETE")
print("="*60)
print("\nNext steps:")
print("  1. Use exported JSON for ML model training")
print("  2. Integrate with SLAM algorithms")
print("  3. Build 3D reconstruction pipeline")
print("  4. Train spatial reasoning models")